In [ ]:
# !conda init bash
# !source ~/.bashrc
# !conda --version
!conda activate LLM
# !conda env list
# print("finish")

In [ ]:
!pip install tensorboard

In [7]:
!pip install --upgrade torch

ERROR: ld.so: object '/usr/lib64/libstdc++.so.6' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/usr/lib64/libstdc++.so.6' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "microsoft/Phi-3-mini-4k-instruct"
# model_name = "Qwen/Qwen2.5-0.5B"
# model_name = "Qwen/Qwen2.5-0.5B-instruct"

# model_name = "KrithikV/MedMobile"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = False)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code = False, torch_dtype=torch.bfloat16)

print(model.config.torch_dtype)
print(model.dtype)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("cuda available:", torch.cuda.is_available())
print("device:", device)

print(model.config.torch_dtype)
print(model.dtype)
print(model.name_or_path)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 13.85it/s]


torch.bfloat16
torch.bfloat16
cuda available: True
device: cuda
torch.bfloat16
torch.bfloat16
microsoft/Phi-3-mini-4k-instruct


In [ ]:
import torch

# Check PyTorch memory usage
print(f"Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GiB")
print(f"Reserved: {torch.cuda.memory_reserved() / 1024**3:.2f} GiB")
print(f"Max Allocated: {torch.cuda.max_memory_allocated() / 1024**3:.2f} GiB")
print(f"Max Reserved: {torch.cuda.max_memory_reserved() / 1024**3:.2f} GiB")

In [ ]:
# print(tokenizer)
chat = [
  {"role": "user", "content": "Hello, how are you?"},
  {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
  {"role": "user", "content": "I'd like to show off how chat templating works!"},
]

print(tokenizer.apply_chat_template(chat, add_generation_prompt = True, tokenize=False))
inputs = tokenizer.apply_chat_template(chat, add_generation_prompt = True, tokenize = True, return_tensors = "pt").to(device)
print("inputs:", inputs)
outputs = model.generate(inputs, max_new_tokens = 512, do_sample = True, temperature = 0.7)
text = tokenizer.batch_decode(outputs)[0]
print("outputs:", text)

In [3]:

from datasets import load_dataset

# chat = [
#     {"role": "user", "content": "Can you format the answer in JSON?"},
#     {"role": "assistant", "content": '{"name": "'},
# ]
# print(tokenizer.apply_chat_template(chat, add_generation_prompt = False, tokenize = False, continue_final_message = True))

ultraMedical = load_dataset("TsinghuaC3I/UltraMedical")

data_list = ultraMedical["train"]
print("done")

In [4]:

def convert_to_chat_format(conversations, is_input):
    chat = []
    for conversation in conversations:
        if conversation["from"] == "human":
            for option_letter in ["A", "B", "C", "D"]:
                conversation["value"] = conversation["value"].replace(f"{option_letter}.", f"[{option_letter}] :")
            chat.append({"role": "user", "content": conversation["value"]})
        elif is_input == False and conversation["from"] == "gpt":
            for option_letter in ["A", "B", "C", "D"]:
                conversation["value"] = conversation["value"].replace(f"{option_letter}.", f"[{option_letter}].")
            chat.append({"role": "assistant", "content": conversation["value"]})
    return chat

In [5]:
# Prepare data from UltraMedical for model to train

# TODO: DataLoader

first_N_data = data_list.select(range(0, 100))

MAX_LENGTH = 1024

# conversations = [{'from': 'human', 'value': 'A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?\n\nA. Ampicillin\nB. Ceftriaxone\nC. Doxycycline\nD. Nitrofurantoin'}, {'from': 'gpt', 'value': "The clinical scenario suggests that the pregnant woman is likely experiencing a urinary tract infection (UTI), given her symptoms of burning upon urination. During pregnancy, the risk of UTIs increases due to physiological changes, and it is important to treat them to prevent complications such as pyelonephritis or preterm labor.\n\nWhen considering antibiotic therapy for a UTI in a pregnant woman, it's crucial to choose an agent that is safe for both the mother and the fetus. Let's evaluate the options:\n\nA. Ampicillin - This is a penicillin antibiotic that is generally considered safe during pregnancy and can be used to treat UTIs. However, it may not be the first choice due to resistance patterns.\n\nB. Ceftriaxone - This is a cephalosporin antibiotic that is also generally safe in pregnancy and effective against a broad range of bacteria. It is typically used for more complicated infections or when other antibiotics are not suitable.\n\nC. Doxycycline - This is a tetracycline antibiotic that is contraindicated during pregnancy because it can affect fetal bone growth and discoloration of teeth.\n\nD. Nitrofurantoin - This antibiotic is commonly used to treat uncomplicated UTIs and is considered safe during most of the pregnancy. However, it is not recommended at the very end of pregnancy (after 38 weeks) due to the risk of hemolytic anemia in the newborn.\n\nGiven that the patient is at 22 weeks gestation and has an uncomplicated UTI, the best choice would be an antibiotic that is safe and commonly used for UTIs in pregnancy.\n\nSo, the answer is D. Nitrofurantoin."}]

# TODO: convert training data to test format:
# inputs: <|user|> <question>\n\n[A] : xxx\n[B] : xxx\n[C] : xxx\n[D] : xxx\n<|end|><|assistant|>
# outputs: <inputs> <answer> ... the answer is [A].<|end|>


def convert_to_tokenized_training_data(data):
    #conversations_list = data["conversations"] # [[{"from":"human", "value":xxx}, {"from":"gpt", "value":xxx}], [{"from":"human", "value":xxx}, {"from":"gpt", "value":xxx}], ...]

    questions = convert_to_chat_format(data["conversations"], is_input = True) #[convert_to_chat_format(conversations, is_input = True) for conversations in conversations_list]
    template_questions = tokenizer.apply_chat_template(questions, tokenize = False, add_generation_prompt = True)
    
    questions_and_answers = convert_to_chat_format(data["conversations"], is_input = False) #[convert_to_chat_format(conversations, is_input = False) for conversations in conversations_list]
    template_questions_and_answers = tokenizer.apply_chat_template(questions_and_answers, tokenize = False, add_generation_prompt = False)

    # print("template_questions", '\n', template_questions, '\n------')
    # print("template_expected_outputs", '\n', template_questions_and_answers, '\n-----')


    tokenized_questions = tokenizer(template_questions, padding = False, truncation=True, max_length=MAX_LENGTH)
    tokenized_inputs = tokenizer(template_questions_and_answers, padding=False, truncation=True, max_length=MAX_LENGTH)
    # print(tokenized_inputs)
    # tokenized_inputs["labels"] = [ids[1:] + [tokenizer.pad_token_id] for ids in tokenized_inputs["input_ids"]] # Shifted inputs as labels
    # tokenized_inputs["labels"] = tokenized_inputs["input_ids"][:]  # Set the target tokens as the 'labels'

    # print("tokenized_questions", '\n', tokenized_questions, '\n------')
    # print("tokenized_inputs", '\n', tokenized_inputs, '\n-----')

    questions_input_ids = tokenized_questions["input_ids"]
    all_content_input_ids = tokenized_inputs["input_ids"]
    tokenized_labels = [-100] * len(questions_input_ids) + all_content_input_ids[len(questions_input_ids):]

    # print("tokenized_labels", '\n', tokenized_labels, '\n-----')

    tokenized_inputs["labels"] = tokenized_labels
    
    padding_num = MAX_LENGTH - len(tokenized_inputs["input_ids"])
    if padding_num > 0:
        tokenized_inputs["input_ids"] = [tokenizer.eos_token_id] * padding_num + tokenized_inputs["input_ids"]
        tokenized_inputs["attention_mask"] = [0] * padding_num + tokenized_inputs["attention_mask"]
        tokenized_inputs["labels"] = [tokenizer.eos_token_id] * padding_num + tokenized_inputs["labels"]
    
    # print(len(tokenized_inputs["input_ids"]), len(tokenized_inputs["labels"]))

    # tokenized_inputs["labels"] = tokenized_inputs["input_ids"][:]
    # question_start_index = len(tokenized_questions["input_ids"])
    # tokenized_inputs["labels"]
    
    # print(tokenized_inputs)
    # print(len(tokenized_inputs["input_ids"]), len(tokenized_inputs["attention_mask"]), len(tokenized_inputs["labels"]))

    tokenized_inputs = {k: torch.tensor(v) for k, v in tokenized_inputs.items()}

    # del tokenized_inputs["attention_mask"]

    # print(tokenized_inputs)
    
    return tokenized_inputs
    

# print(first_N_data[0])
# tokenized_training_data = convert_to_tokenized_training_data(first_N_data[0])
tokenized_training_data = first_N_data.map(convert_to_tokenized_training_data, batched = False)

print("done")
# print(tokenized_training_data)
# print("---")
# print(tokenizer.decode(tokenized_training_data[0]["input_ids"], skip_special_tokens = False))
# print("---")
# print(tokenizer.decode(tokenized_training_data[0]["labels"], skip_special_tokens = False)) # skip_special_tokens = False will show <|endoftext|><|user|><|end|><|assistant|>


done


In [6]:
# Train
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "./Documents/fine_tuned_model_checkpoints",
    save_strategy = "no", # TODO, now don't save checkpoints #"epoch",
    per_device_train_batch_size = 1, # As specified in the paper: batch_size: 32
    num_train_epochs = 3, # As specified in the paper: 3 epochs
    learning_rate = 1e-4, # As specified in the paper: 1e-4
    # save_total_limit = 1, # TODO, now keep only the last checkpoint
    fp16 = True, # TODO
    logging_steps = 1,
    logging_dir = "./Documents/logs",  # Directory for logs
    # report_to = ["tensorboard"],  # Enable logging to TensorBoard
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_training_data
)

trainer.train()

import shutil
shutil.rmtree("/home/sheju347/Documents/fine_tuned_model/", ignore_errors=True)
print ("previous model removed.")

# Save the final model
output_dir = "./Documents/fine_tuned_model"

model.to(torch.bfloat16)  # Convert to bfloat16
model.save_pretrained(output_dir, 
                        # save_function=torch.save,  # Use standard PyTorch save
                        # state_dict=model.state_dict(),  # Only save the model weights
                        # safe_serialization=True,  # More efficient serializationsave_optimizer_state=False
                     )
tokenizer.save_pretrained(output_dir,
                         # legacy_format=False  # Use newer, more efficient format
                     )

ERROR: ld.so: object '/usr/lib64/libstdc++.so.6' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.


OutOfMemoryError: CUDA out of memory. Tried to allocate 18.00 MiB. GPU 0 has a total capacity of 79.10 GiB of which 2.69 MiB is free. Process 1499667 has 75.44 GiB memory in use. Including non-PyTorch memory, this process has 3.64 GiB memory in use. Of the allocated memory 3.14 GiB is allocated by PyTorch, and 1.84 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Check fine-tuned model
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# model_name = "./code/Documents/fine_tuned_model"
model_name = "./Documents/fine_tuned_model"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = False)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code = False)

print(model.config.torch_dtype)
print(model.dtype)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("cuda available:", torch.cuda.is_available())
print("device:", device)

print(model.config.torch_dtype)
print(model.dtype)
print(model.name_or_path)


In [ ]:
chat = [
    {"role": "user", "content": "Can you format the answer in JSON?"},
    {"role": "assistant", "content": '{"name": "'},
]

print(tokenizer.apply_chat_template(chat, add_generation_prompt = False, tokenize = False, continue_final_message = True))

In [ ]:
conversations = ultraMedical["train"][0]["conversations"]
chat = convert_to_chat_format(conversations, is_input = True)
print(chat)

In [ ]:
# Test apply_chat_template // https://huggingface.co/docs/transformers/main/en/chat_templating

print(model.name_or_path)

def format_choices(choices):
    a = zip(list(choices.keys()), choices.values())
    final_answers = []
    for x,y in a:
        final_answers.append(f'[{x}] : {y}')
    return "\n".join(final_answers)


def run_inference(content, model, tokenizer, max_new_tokens, temperature):
    messages = [{"role": "user", "content": f"{content}"}]
    # add_generation_prompt indicates the start of a response
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt = True, return_tensors = "pt").to(device)
    # print("inputs:", tokenizer.apply_chat_template(messages, add_generation_prompt = True, tokenize = False))
    outputs = model.generate(inputs, max_new_tokens = max_new_tokens, do_sample = True, temperature = temperature)
    text = tokenizer.batch_decode(outputs)[0]
    return text

prompt = f'''
{{question}} \n
{{choices}}
'''

# Training data index 0 (GBaker/MedQA-USMLE-4-options)
question = "A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?"
choices = {
"A": "Ampicillin",
"B": "Ceftriaxone",
"C": "Doxycycline",
"D": "Nitrofurantoin"
}

# Training data index 1
# question = "A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy. Which of the following precautions could have prevented the death of the baby?"
# choices = {
# "A": "Placing the infant in a supine position on a firm mattress while sleeping",
# "B": "Keeping the infant covered and maintaining a high room temperature",
# "C": "Application of a device to maintain the sleeping position",
# "D": "Avoiding pacifier use during sleep"
# }

# Training data index 1
# question = "A mother brings her 3-week-old infant to the pediatrician's office because she is concerned about his feeding habits. He was born without complications and has not had any medical problems up until this time. However, for the past 4 days, he has been fussy, is regurgitating all of his feeds, and his vomit is yellow in color. On physical exam, the child's abdomen is minimally distended but no other abnormalities are appreciated. Which of the following embryologic errors could account for this presentation?"
# choices = {
# "A": "Abnormal migration of ventral pancreatic bud",
# "B": "Complete failure of proximal duodenum to recanalize",
# "C": "Abnormal hypertrophy of the pylorus",
# "D": "Failure of lateral body folds to move ventrally and fuse in the midline"
# }

# Test data index 0
# question = "A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?"
# choices = {
# "A": "Disclose the error to the patient and put it in the operative report",
# "B": "Tell the attending that he cannot fail to disclose this mistake",
# "C": "Report the physician to the ethics committee",
# "D": "Refuse to dictate the operative report"
# }

formated_choices = format_choices(choices)

model_prompt = prompt.format(question = question, choices = formated_choices)

# print(model_prompt)

output_text = run_inference(model_prompt, model, tokenizer, max_new_tokens = 1024, temperature = 0.7)
# output_text = output_text.split("<|assistant|>")[-1]
print(output_text)

In [ ]:
print ("FOR TEST")

print ("model: ", model.name_or_path)

# prompt = """
# Question: A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?
# Choices:
# A: Disclose the error to the patient but leave it out of the operative report
# B: Disclose the error to the patient and put it in the operative report
# C: Tell the attending that he cannot fail to disclose this mistake
# D: Report the physician to the ethics committee
# E: Refuse to dictate the operative report
# Given five answer candidates, A, B, C, D, and E, choose the best answer choice.
# The answer is:
# """

question = "A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?\n\nA. Ampicillin\nB. Ceftriaxone\nC. Doxycycline\nD. Nitrofurantoin"
# Respond only with letter A, B, C, D, E. Don't need other information.

prompt = f"Give a proper answer to the question:\nquestion: {question}\n\nThe answer is: "

# prompt = """

print ("Prompt: ", prompt)
print ("--------------")

import time
start_time = time.time()

inputs = tokenizer(prompt, return_tensors = "pt", padding=False).to(device)
with torch.no_grad(): # NO gradient calculation for inference
    output = model.generate(
        inputs["input_ids"],
        max_new_tokens = 1024,
        attention_mask=inputs["attention_mask"],  # Use attention mask
        pad_token_id=tokenizer.eos_token_id,  # Set pad token ID
        # use_cache=False,
        #do_sample = False
    )

answer = tokenizer.decode(output[0], skip_special_tokens = True)

# print(len(answer))
# print("original decode answer:", answer)

answer = answer.replace(prompt, '').strip()

finish_time = time.time()
elapse_time = finish_time - start_time
print("elapse_time: ", elapse_time)
print ("--------------")

print ("Answer: ", answer)


In [ ]:
# Train model on "TsinghuaC3I/UltraMedical" dataset

import shutil
shutil.rmtree("/home/sheju347/Documents/fine_tuned_model/", ignore_errors=True)
print ("previous model removed.")

from datasets import load_dataset
from transformers import Trainer, TrainingArguments

ultraMedical = load_dataset("TsinghuaC3I/UltraMedical")

data_list = ultraMedical["train"]
print(len(data_list), type(data_list))

# print((data_list[0:10]))
first_N_data = data_list.select(range(0, 10000)) # TODO

print(type(first_N_data))

def preprocess_function(data):
    print("data:", data)
    # input_text = [f"Give a proper answer to the question:\nquestion: {conversation[0]['value']}\n\nThe answer is: " for conversation in data["conversations"]] # (context) + question + options
    # target_text = [f"{conversation[1]['value']}" for conversation in data["conversations"]]  # CoT + final answer
    # # target_text = [f"A" for conversation in data["conversations"]]  # CoT + final answer (TODO)
    
    input_text = [f"Give a proper answer to the question:\nquestion: {conversation[0]['value']}\n\nThe answer is: "
                      for conversation in data['conversations']] # (context) + question + options
    target_text = [f"Give a proper answer to the question:\nquestion: {conversation[0]['value']}\n\nThe answer is: D. \nExplanation: {conversation[1]['value']}<|end>"
                      for conversation in data["conversations"]]  # CoT + final answer


    # print("input:", input_text)
    print("target:", target_text)
    
    # prompt = f"Give a proper answer to the question:\nquestion: {question}\n\nThe answer is: "

    inputs = tokenizer(input_text, padding="max_length", truncation=True, max_length=1024)
    targets = tokenizer(target_text, padding="max_length", truncation=True, max_length=1024)
    inputs["labels"] = targets["input_ids"]  # Set the target tokens as the 'labels'

    return inputs

    # prompt = input_text
    # data['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    # data['labels'] = tokenizer(target_text, padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    # return data

tokenized_train = first_N_data.map(preprocess_function, batched=True)

training_args = TrainingArguments(
    output_dir = "./Documents/fine_tuned_model_checkpoints",
    save_strategy = "no", # TODO, now don't save checkpoints #"epoch",
    per_device_train_batch_size = 4, # As specified in the paper: batch_size: 32
    num_train_epochs = 3, # As specified in the paper: 3 epochs
    learning_rate = 1e-4, # As specified in the paper: 1e-4
    # save_total_limit = 1, # TODO, now keep only the last checkpoint
    fp16 = True, # TODO
    logging_steps = 1,
    logging_dir = "./Documents/logs",  # Directory for logs
    
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train
)

trainer.train()

# Save the final model
output_dir = "./Documents/fine_tuned_model"

model.to(torch.bfloat16)  # Convert to bfloat16
model.save_pretrained(output_dir, 
                        # save_function=torch.save,  # Use standard PyTorch save
                        # state_dict=model.state_dict(),  # Only save the model weights
                        # safe_serialization=True,  # More efficient serializationsave_optimizer_state=False
                     )
tokenizer.save_pretrained(output_dir,
                         # legacy_format=False  # Use newer, more efficient format
                     )